### 데이터 불러 오기

In [1]:
import glob
import os 
import numpy as np


# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

 There must be some kind of way outta here
Said the joker to the thief
There's too much confusion
I can't get no relief Business men, they drink my wine
Plowman dig my earth
None were level on the mind
Nobody up at his word
Hey, hey No reason to get excited
The thief he kindly spoke
There are many here among us


### 데이터 전처리

In [3]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue

    corpus.append(preprocess_sentence(sentence))

corpus[:100]
len(corpus)

175986

### 데이터 토큰화 

In [5]:
import tensorflow as tf

# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=20, padding='post')
    
    print(tensor,'\n', tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

len(tensor), len(corpus)

[[  2  64 280 ...   0   0   0]
 [  2 107   6 ...   0   0   0]
 [  2  64  16 ...   0   0   0]
 ...
 [  2  75  46 ...   0   0   0]
 [  2  49   4 ...   0   0   0]
 [  2  13 640 ...   0   0   0]] 


(175986, 175986)

In [6]:
print(tensor[:10, :12])

[[   2   64  280   27   99  528   19   85  779   93    3    0]
 [   2  107    6 6490   10    6 2478    3    0    0    0    0]
 [   2   64   16  104  178 2966    3    0    0    0    0    0]
 [   2    5   32   15   44   38 5344  783  622    4   45  470]
 [   2    1  993   13  672    3    0    0    0    0    0    0]
 [   2  891  176 1978   18    6  169    3    0    0    0    0]
 [   2  238   29   71  105  475    3    0    0    0    0    0]
 [   2  135    4  135   38  566   10   44 2839    3    0    0]
 [   2    6 2478   55 5044 1765    3    0    0    0    0    0]
 [   2   64   77  302   93 2186  131    3    0    0    0    0]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


### 데이터 분할

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  64 280  27  99 528  19  85 779  93   3   0   0   0   0   0   0   0
   0]
[ 64 280  27  99 528  19  85 779  93   3   0   0   0   0   0   0   0   0
   0]


In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, enc_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 19)
Target Train: (140788, 19)


In [10]:
BUFFER_SIZE = len(src_input)
print(BUFFER_SIZE)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

175986


<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

### 텍스트 모델 생성

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 12001), dtype=float32, numpy=
array([[[ 5.77662722e-05, -2.01872288e-04,  6.88908549e-05, ...,
          4.32757042e-05,  8.33604718e-05,  3.94492636e-05],
        [ 4.45137630e-05, -2.45874369e-04,  1.01058649e-04, ...,
         -1.48748913e-05,  8.63156893e-05,  1.08451532e-04],
        [ 7.53570130e-05, -1.66137164e-04,  1.08017863e-04, ...,
         -2.39388813e-04,  7.54687499e-05,  6.62839520e-05],
        ...,
        [ 8.71183467e-04,  8.01534450e-04,  2.50482495e-04, ...,
         -2.35191255e-04,  5.55614475e-04, -1.10973662e-03],
        [ 1.16467557e-03,  9.28541645e-04,  3.03810055e-04, ...,
         -2.07216843e-04,  5.53928141e-04, -1.31371943e-03],
        [ 1.44003297e-03,  1.04449771e-03,  3.41374776e-04, ...,
         -1.73404929e-04,  5.59022068e-04, -1.51017925e-03]],

       [[ 5.77662722e-05, -2.01872288e-04,  6.88908549e-05, ...,
          4.32757042e-05,  8.33604718e-05,  3.94492636e-05],
        [-2.62575013e-05, -4.87911690e-04, -6

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
687/687 [==============================] - 282s 406ms/step - loss: 3.3710
Epoch 2/10
687/687 [==============================] - 281s 409ms/step - loss: 2.5077
Epoch 3/10
687/687 [==============================] - 287s 417ms/step - loss: 2.3438
Epoch 4/10
687/687 [==============================] - 279s 406ms/step - loss: 2.2285
Epoch 5/10
687/687 [==============================] - 281s 408ms/step - loss: 2.1321
Epoch 6/10
687/687 [==============================] - 280s 408ms/step - loss: 2.0471
Epoch 7/10
687/687 [==============================] - 280s 408ms/step - loss: 1.9647
Epoch 8/10
504/687 [=====================>........] - ETA: 1:14 - loss: 1.8902

### 모델 성능 평가 

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> miss you", max_len=20)

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i want", max_len=20)

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i cannot wait", max_len=20)

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> merry me", max_len=20)

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> kiss me", max_len=20)

### 느낀 점 

#### 앞으로 한국어 가사로 한번 만들어 볼 생각이다. 
#### 전처럼 막 실행 해 볼 수 없는 처음으로 시간에 대해 고민해 보게 만드는 프로젝트 였다.